In [ ]:
# list of non-food categories
nonfood_list = ['Cleaningsupplies','Clothing/accessories','Electronics',\
                'Flower','Kitchentool','Officesupplies','Paperproduct',\
                'Paperproducts','Petfood','petfood','Plasticproduct',\
                'plasticproduct','Supplement','Toiletries',\
                'toiletries','Utensil','utensil']

# drops non-food items
data.drop(data[data['FoodCategory'].isin(nonfood_list)].index, inplace=True)

In [ ]:
# drops participants with less than 50 data points
low_part = data['ID'].value_counts() < 50
for p in low_part[low_part == True].index:
    data.drop(data[data['ID'] == p].index, inplace=True)

In [ ]:
# crude first pass interpolation of missing dates
# if 20% or more data is missing abort
# otherwise attribute all missing dates to the middle of the largest date gap
for id in data['ID'].unique():
    # 20% NaN check
    nan_count = sum(data[data['ID'] == id]['RecDate'].isna())
    item_count = len(data[data['ID'] == id]['RecDate'])
    nan_percent = nan_count / item_count
    if nan_percent <= .2:
        # interpolate to middle of largest gap in dates
        shopping_days = data[data['ID'] == id]['RecDate'].unique()
        shopping_days.sort()
        gaps = []
        for d in range(len(shopping_days)-1):
            gaps.append(shopping_days[d+1] - shopping_days[d])
        ind = gaps.index(max(gaps))
        interp_date = (shopping_days[ind] + max(gaps)/2).astype('datetime64[D]')
        data.loc[(data['ID'] == 113) & pd.isnull(data['RecDate']), 'RecDate'] = interp_date
    else:
        continue

In [ ]:
# remove (modifier) data from 'Item (modifier)' column
paren = re.compile(r' \(.*\)')
#[paren.findall(x) for x in data['Item (modifier)']]
data['Item'] = [re.sub(paren,'', str(x)) for x in data['Item (modifier)']]
# df[col] = df[col].str.replace(r'\(|\)|\?', '', regex = True)

In [ ]:
# Spelling correction on Item
from textblob import TextBlob
df_max['ItemSpellchecked'] = df_max['Item'].apply(lambda item: ''.join(TextBlob(item).correct()))